In [1]:
import pandas as pd
import numpy as np
import nltk
import re
nltk.download('stopwords')
nltk.download('punkt')
import os.path
from nltk.tag import StanfordNERTagger
from nltk.tokenize import word_tokenize

In [7]:
lst_commentaries = []
lst_reports = []
for i in range(1,700):
    sent = ''
    if (os.path.isfile(r'C:\Users\sachi\Downloads\NCI\Thesisi\Project\CSV\commentary_{0}.csv'.format(i))) and (os.path.isfile(r'C:\Users\sachi\Downloads\NCI\Thesisi\Project\Reports\short_reports\report_{0}.txt'.format(i))):
        df = pd.read_csv(r'C:\Users\sachi\Downloads\NCI\Thesisi\Project\CSV\commentary_{0}.csv'.format(i))
        with open(r'C:\Users\sachi\Downloads\NCI\Thesisi\Project\Reports\short_reports\report_{0}.txt'.format(i), 'r') as f:
            lst_reports.append(f.readlines()[0])
        df.drop(columns=['Unnamed: 0'], axis = 1, inplace=True)
        for j in range(df.shape[0]):
            sent = sent + df.iloc[j,0]+'.'
        lst_commentaries.append(sent)

SyntaxError: invalid syntax (<ipython-input-7-9a61594d74de>, line 7)

In [4]:
data_frame = pd.DataFrame({'input_text':lst_commentaries, 'target_text':lst_reports})

NameError: name 'lst_commentaries' is not defined

In [5]:
data_frame.shape

(465, 2)

In [6]:
# Dataset without masking the data
data_frame.to_excel('data_2_withoutMask.xlsx')

### Cosine Similarity

In [8]:
st = StanfordNERTagger(r'C:\Users\sachi\Downloads\stanford-ner-4.2.0\stanford-ner-4.2.0\stanford-ner-2020-11-17\classifiers\english.all.3class.distsim.crf.ser.gz',
  r'C:\Users\sachi\Downloads\stanford-ner-4.2.0\stanford-ner-4.2.0\stanford-ner-2020-11-17\stanford-ner.jar',
  encoding='utf-8')

In [9]:
dict = {}
for i in range(data_frame.shape[0]):
    text = data_frame.iloc[i,0]
    tokenized_text = word_tokenize(text)
    classified_text = st.tag(tokenized_text)

    list_of_person = [i for i in classified_text if i[1] == 'PERSON']
    dict[i] = list_of_person

In [10]:
plyr_dict = {}
for z in range(len(dict)):  
    lst_plyr = []

    for i in dict[z]:
        if i[0] not in lst_plyr:
            lst_plyr.append(i[0])

    lst_lyr_to_delete = [ 'Fended.Mahedi', 'Ranjith', 'McGlashan', 'Sreshth', 'Shah', 'Dan', '.Mahedi','P.', 'Marsh.Mahedi', 'Hossain','Mahedi.Mahedi',
 'defend.Mahedi',
 'cover.Mahedi', 'Binoy', 'Kishore','Shashank', 'George', 'Man', 'of', 'Ben','the','Leslie', 'Reifer','Nigel',
 'Duguid', 'Jr','Hope', 'Match','Jr.', 'Penna', 'Della', 'Peter', 'Good','different.Tom','intercepts.Sam',
 'mid-on.Sam',  'Allen.Australia','(',"doesn'.Kane", 'RussSir', 'defended.Tom','Mo','defending.Tom',
 'midwicket.Tom','to', 'der', 'dre','van','Cricinfo',
 'mid-off.Tom',
 'defends.Tom',
 'Ultra-Edge',
 'mid-off.Sam', 'Wade.Starc','Chandan',' Duorah', 'Hemant ','Brar','there.Bumrah','Richardson.Kane',
 'Stoinis.Kane','.Kane','Kohli.Agar','misses.Kane',')','cover.Kane','delivers.Kane','Carey.Kane','blade.Muhammad', 'boundary.Kane',
 'defended.Kane',
 'him.Mohammad','.Muhammad',
 '.Mohammad',
 'Warner.Mohammad',
 'though.Mohammad',
 'it.Mohammad',
 'midwicket.Mohammad',
 'mid-off.Mohammad',
 'away.Mohammad',]
    for j in lst_lyr_to_delete:
        if j in lst_plyr:
            lst_plyr.remove(j)    
    plyr_dict[z] = lst_plyr

In [11]:
plyr_dict_1 = {}
for a in range(len(plyr_dict)):
    l = []
    for k in plyr_dict[a]:
        if '.' not in k:
            l.append(k)
    plyr_dict_1[a] = l

In [12]:
plyr_dict = None
plyr_dict = plyr_dict_1

In [13]:
for i in plyr_dict:
    for j in plyr_dict[i]:
        if len(j) <4:
            plyr_dict[i].remove(j)

In [16]:
# Replacing the player names by Player#
input_text = []
target_text = []
for m in range(data_frame.shape[0]):
    text = data_frame.iloc[m,0]
    summary = data_frame.iloc[m,1]
    for n in range(len(plyr_dict[m])):
             text = text.replace(plyr_dict[m][n],'Player#')
             summary = summary.replace(plyr_dict[m][n],'Player#')
    input_text.append(text)
    target_text.append(summary)

In [17]:
data_frame = pd.DataFrame({'input_text':input_text, 'target_text':target_text})

In [19]:
data_frame.to_excel('data_2.xlsx')

In [30]:
def replace_digits(tex):
    """This function replaces the digits by digit#"""
    j = len(tex)
    i=0
    lst_idx = []
    while True:
            idx = []
            if i < j:
                count = 0
                if ((ord(tex[i]) > 47) & (ord(tex[i]) < 58)):
                    start = i
                    while True:
                        if ((ord(tex[i]) > 47) & (ord(tex[i]) < 58)):
                            count = count + 1
                            i = i+1
                            continue
                        else:
                            idx.append(start)
                            idx.append(i)
                            lst_idx.append(idx)                   
                            i = i -1
                            break
        
                i = i +1
        
            else:
                break
    num = []
    for i in range(len(lst_idx)):
        num.append(tex[lst_idx[i][0]:lst_idx[i][1]])
    
    for i in range(len(num)):
        if num[i] in tex:
            tex = tex.replace(num[i], 'digit#',1)
    
    return tex
    
    

In [31]:
input_text = []
target_text = []
for i in range(data_frame.shape[0]):
    input_text.append(replace_digits(data_frame.iloc[i,0]))
    target_text.append(replace_digits(data_frame.iloc[i,1]))

In [32]:
data_frame = pd.DataFrame({'input_text':input_text, 'target_text':target_text})

In [ ]:
data_frame.to_excel('data_2.xlsx')